In [8]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

from os import listdir
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

from nltk.tokenize import word_tokenize


from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
unprocessed_file_name = 'unprocessed_file.pkl'
cleaned_file_name ='cleaned_file.pkl'
cnn = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/summarisation/cnn/stories"
path_of_downloaded_files = "/Users/rohankilledar/Documents/MSc Artificial Intelligence/repos/Natural Language Processing/glove.6B.50d.txt"
filename = "glove.6B.50d.txt"
embedding_dim = 50
data_size = 2000
hidden_size = 50

In [10]:
glove_file = datapath(path_of_downloaded_files)
word2vec_glove_file = get_tmpfile(filename)
glove2word2vec(glove_file, word2vec_glove_file)
word_vectors = KeyedVectors.load_word2vec_format(word2vec_glove_file)
print('loaded %s word vectors from %s.' % (len(word_vectors.key_to_index),filename ))

<ipython-input-10-0c6d93aa8805>:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_glove_file)


In [ ]:
SOS_token = 0
EOS_token = 1

sos = 'sos'
eos = 'eos'



class Lang:
    def __init__(self, name):
        self.name = name
        self.words = []
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0  # Count SOS and EOS
        self.glove = {}
        self.embedding_dim = word_vectors.get_vector('office').shape[0]

    def addSentence(self, sentence):
        for word in word_tokenize(sentence):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
            self.words.append(word)
            if word not in self.glove:
                self.glove[word] = np.random.normal(scale=0.6 , size= (self.embedding_dim,))
        else:
            self.word2count[word] += 1

    def addPretrained(self,path_of_downloaded_files):
        with open(path_of_downloaded_files) as f:
            for indx,l in enumerate(f):
                line = l.split()
                word = line[0]
                if word == sos:
                    first_word = self.words[0]
                    self.words.append(first_word)
                    self.words[0] = sos
                    self.word2index[first_word] = indx
                    self.word2index[sos] = 0
                    self.index2word[indx] = first_word
                    self.index2word[0] = sos
                    self.glove[sos] = word_vectors[indx]
                    self.glove[first_word] = word_vectors[0]
                    self.word2count[sos] = 1
                    self.n_words += 1

                elif word == eos:
                    sec_word = self.words[1]
                    self.words.append(sec_word)
                    self.words[1] = eos
                    self.index2word[indx] = sec_word
                    self.index2word[1] = eos
                    self.word2index[sec_word] = indx
                    self.word2index[eos] = 1
                    self.glove[eos] = word_vectors[indx]
                    self.glove[sec_word] = word_vectors[1]
                    self.word2count[eos] = 1
                    self.n_words += 1
                else:
                    self.words.append(word)
                    self.word2count[word] = 1
                    self.word2index[word] = indx
                    self.index2word[indx]=word
                    self.glove[word] = word_vectors[indx]
                    self.n_words += 1
                    

In [ ]:
def decontracted(phrase): 
    phrase = re.sub(r"won't", "will not", phrase) 
    phrase = re.sub(r"can\'t", "can not", phrase)  
    phrase = re.sub(r"n\'t", " not", phrase)  
    phrase = re.sub(r"\'re", " are", phrase)  
    phrase = re.sub(r"\'s", " is", phrase)  
    phrase = re.sub(r"\'d", " would", phrase)  
    phrase = re.sub(r"\'ll", " will", phrase)  
    phrase = re.sub(r"\'t", " not", phrase)  
    phrase = re.sub(r"\'ve", " have", phrase)  
    phrase = re.sub(r"\'m", " am", phrase)  
    return phrase

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    
    
    dash_indx = s.find('(CNN) --')
    if dash_indx>=0: #and dash_indx<=20:
        s = s[dash_indx+len('(CNN) --'):]
        
    s=decontracted(s)
    s = unicodeToAscii(s.lower().strip())

    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [ ]:
def read_file(filename):
    file = open(filename, encoding= 'UTF-8')
    text = file.read()
    file.close()
    return text

def split_text(article):
    indx = article.index('@highlight')
    story = article[:indx]
    highlight = article[indx:].split('@highlight')

    highlight = ". ".join([h.strip() for h in highlight if len(h)>0])
    return story,highlight

In [ ]:
def read_all(folder):
    dataset = list()

    for file in tqdm(listdir(folder)):
        filename = folder + '/' + file
        article = read_file(filename)
        story,highlight = split_text(article)

        dataset.append({'story':story, 'highlight':highlight})
    
    return dataset

dataset = read_all(cnn)

  0%|          | 0/92579 [00:00<?, ?it/s]

In [ ]:
#saving dataset for cleaning
import pickle
output_file = open(unprocessed_file_name,'wb')
pickle.dump(dataset, output_file)
output_file.close()

In [ ]:

dataset = pd.read_pickle(unprocessed_file_name)
#reducing the dataset for initial testing of model
dataset = dataset[:data_size]

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df['story'] = df['story'].apply(lambda x: normalizeString(x))
df['highlight'] = df['highlight'].apply(lambda x: normalizeString(x))

In [ ]:
df['highlight'] = df['highlight'].apply(lambda x: sos + " " + x )

In [ ]:
df['word_count_text'] = df['story'].apply(lambda x: len(str(x).split()))
df['highlight_count'] = df['highlight'].apply(lambda x: len(str(x).split()))
from math import floor
print("The mean word count length of text article is " + str(floor(df['word_count_text'].mean())))
print("The mean word count length of summary/highlight is " + str(floor(df['highlight_count'].mean())))

max_article_len = floor(df['word_count_text'].max())
max_summary_len = floor(df['highlight_count'].max())
print("The max word count length of text article is " + str(max_article_len))
print("The max word count length of summary/highlight is " + str(max_summary_len))

The mean word count length of text article is 689
The mean word count length of summary/highlight is 46
The max word count length of text article is 1884
The max word count length of summary/highlight is 82


In [ ]:
df.drop('word_count_text', axis=1, inplace=True)
df.drop('highlight_count', axis=1, inplace=True)

In [ ]:
with open("processed_data.pkl","wb") as save_path:
    pickle.dump(df, save_path)

In [ ]:
vocab = Lang('vocab')
vocab.addPretrained(path_of_downloaded_files)
def prepareData():
    for indx,row in tqdm(df.iterrows()):
        vocab.addSentence(row['story'])
        vocab.addSentence(row['highlight'])
    print("counted Words:")
    print(vocab.name, vocab.n_words)
prepareData()

0it [00:00, ?it/s]

counted Words:
vocab 402650


In [ ]:
with open("vocab.pkl","wb") as save_path:
    pickle.dump(df, save_path)

In [ ]:
pairs = df.values.tolist()

In [ ]:
def indexesFromSentence(lang, sentence):
    # return [lang.word2index[word] for word in sentence.split(' ')]
    return [lang.word2index[word] for word in word_tokenize(sentence)]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(vocab, pair[0])
    target_tensor = tensorFromSentence(vocab, pair[1])
    return (input_tensor, target_tensor)

In [ ]:

def generate_weights(unique_word_corpus,glove, tens=True):
    matrix_len = len(unique_word_corpus)
    unique_word2indx = {}
    
    weight_matrix  = np.zeros((matrix_len, embedding_dim))
   
    words_found = 0

    for indx, word in enumerate(unique_word_corpus):
        try:
            unique_word2indx[word] = indx
            weight_matrix[indx] = glove[word]
            words_found +=1
        except KeyError:
            weight_matrix[indx] = np.random.normal(scale=0.6 , size= (embedding_dim,))
            unique_word2indx[word] = indx
    if tens:
        return torch.from_numpy(weight_matrix) , unique_word2indx
    else:
        return weight_matrix,unique_word2indx

weight_matrix, meh= generate_weights(vocab.words, vocab.glove)

In [ ]:
weight_matrix

tensor([[ 0.5139, -0.7162, -0.2702,  ...,  0.1683,  0.5761,  0.0467],
        [-0.3090,  0.1448,  1.1682,  ..., -0.4097, -0.4651, -0.2958],
        [ 0.1516,  0.3018, -0.1676,  ..., -0.3565,  0.0164,  0.1022],
        ...,
        [-0.7758, -0.6933,  0.5565,  ...,  0.2769, -0.9246, -0.0116],
        [-0.0579,  0.5945, -0.5711,  ..., -0.3212,  0.7856,  1.3458],
        [ 0.2870, -0.0951, -1.0401,  ..., -0.2164,  0.4428, -0.0370]],
       dtype=torch.float64)

In [ ]:
vocab.glove['sos']

array([ 0.51387 , -0.71619 , -0.27022 , -0.39605 ,  0.077725, -0.51019 ,
       -0.096037, -0.51926 ,  1.1398  , -0.48839 ,  0.55519 , -0.079659,
        0.63065 , -0.19989 , -0.72763 , -1.1698  , -0.78838 , -0.11933 ,
       -0.016341,  0.58433 , -0.60493 ,  0.34394 , -0.10728 ,  0.38154 ,
        0.60439 ,  0.098549, -0.62968 , -0.55329 , -0.40243 , -0.75822 ,
        0.58165 ,  0.8636  , -0.80085 ,  0.26178 , -1.1834  ,  0.099697,
        0.56937 , -1.6251  ,  0.4834  ,  0.36138 ,  0.73861 , -0.65523 ,
       -0.31002 , -0.65169 ,  0.51786 ,  0.39711 ,  0.22438 ,  0.16832 ,
        0.57609 ,  0.046694], dtype=float32)

In [ ]:
word = 'the'
embeds = nn.Embedding(vocab.n_words, embedding_dim)  # 2 words in vocab, 5 dimensional embeddings
embeds =embeds.from_pretrained(weight_matrix)
lookup_tensor = torch.tensor([vocab.word2index[word]], dtype=torch.long)
print(vocab.word2index[word])
print(lookup_tensor)
hello_embed = embeds(lookup_tensor)
print(hello_embed)
print(weight_matrix[vocab.word2index[word]])


28069
tensor([28069])
tensor([[ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
         -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
          2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9.5095e-03,
          1.1658e-02,  1.0204e-01, -1.2792e-01, -8.4430e-01, -1.2181e-01,
         -1.6801e-02, -3.3279e-01, -1.5520e-01, -2.3131e-01, -1.9181e-01,
         -1.8823e+00, -7.6746e-01,  9.9051e-02, -4.2125e-01, -1.9526e-01,
          4.0071e+00, -1.8594e-01, -5.2287e-01, -3.1681e-01,  5.9213e-04,
          7.4449e-03,  1.7778e-01, -1.5897e-01,  1.2041e-02, -5.4223e-02,
         -2.9871e-01, -1.5749e-01, -3.4758e-01, -4.5637e-02, -4.4251e-01,
          1.8785e-01,  2.7849e-03, -1.8411e-01, -1.1514e-01, -7.8581e-01]],
       dtype=torch.float64)
tensor([ 4.1800e-01,  2.4968e-01, -4.1242e-01,  1.2170e-01,  3.4527e-01,
        -4.4457e-02, -4.9688e-01, -1.7862e-01, -6.6023e-04, -6.5660e-01,
         2.7843e-01, -1.4767e-01, -5.5677e-01,  1.4658e-01, -9

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layer = 1

       # self.num_layers = num_layers
        self.embedding = nn.Embedding(num_embeddings = vocab.n_words,embedding_dim= embedding_dim)
        self.embedding = self.embedding.from_pretrained(weight_matrix)
        #self.embedding.from_pretrained(glove)
        self.lstm = nn.LSTM(input_size = hidden_dim, hidden_size = hidden_dim) #, num_layers = 1, bidirectional = True, batch_first = True)
        #self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):

        embedded = self.embedding(input).view(1,1,-1)
        output = embedded.float()
       # print(output)
        #print(hidden)
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        #in case of LSTM we need h0 and c0 hence init this as a tuple (h0,c0) and passed as hidden to lstm and in case of gru its just h0
       return (torch.zeros(self.num_layer, 1, self.hidden_size, dtype=torch.float, device=device), torch.zeros(self.num_layer,1,self.hidden_size,dtype = torch.float, device= device))

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(num_embeddings= output_size,embedding_dim= hidden_size)
        self.embedding.from_pretrained(weight_matrix)
        self.lstm = nn.LSTM(input_size= hidden_size, hidden_size= hidden_size) #, num_layers = 1, bidirectional = False, batch_first = True)
        #self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self.fc = nn.Linear(in_features= hidden_size , out_features=vocab.n_words)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = F.relu(embedded)
        
        #hidden is [2,1,50]
        #emb is [1,1,50]
        output, hidden = self.lstm(embedded, hidden)
        #output = self.softmax(self.out(output[0]))
        output = self.fc(output[0])
        
        output = self.softmax(output)
        return output, hidden


In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=max_article_len):
        super(AttnDecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.embedding.from_pretrained(weight_matrix)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
       
    
        AttnDecoderRNN.embedded = embedded
        
        AttnDecoderRNN.hidden = hidden
        AttnDecoderRNN.encoder_outputs = encoder_outputs
        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0].view(1,-1)), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

                                 

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.lstm(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
       return (torch.zeros(1, 1, self.hidden_size, dtype=torch.int64, device=device), torch.zeros(1,1,self.hidden_size,dtype = torch.int64, device= device))

In [ ]:
# from nltk.tokenize import word_tokenize

# def indexesFromSentence(lang, sentence):
#     #return [lang.glove[word] for word in sentence.split(' ')]
#     return [lang.word2index[word] for word in word_tokenize(sentence)]
#     #return [lang.word2index[word] for word in sentence.split(' ')]
# def tensorFromSentence(lang, sentence):
#     indexes = indexesFromSentence(lang, sentence)
#     indexes.append(EOS_token)
#     # indexes.append(lang.glove[eos])
#     #change it to include glove values
#     #return indexes
#     return torch.Tensor(indexes,dtype=torch.long).view(-1, 1)


# def tensorsFromPair(pair):
#     input_tensor = tensorFromSentence(vocab, pair[0])
#     target_tensor = tensorFromSentence(vocab, pair[1])
#     return (input_tensor, target_tensor)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, attn, max_length=vocab.n_words):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            if attn :
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden= decoder(
                    decoder_input, decoder_hidden)
                #tar_tens = weight_matrix[target_tensor[di].item()].view(-1).float()
            loss += criterion(decoder_output, target_tensor[di])
            
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            if attn :
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
            else:
                decoder_output, decoder_hidden= decoder(
                    decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            tar_tens = weight_matrix[target_tensor[di].item()].view(-1).float()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, attn=False, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    if attn:
        print("training with attention.")
    print(start)

    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print('encoder optimizer init')
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    print('decoder optimizer init')
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    print('choosing training pairs:' + str(n_iters))
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    
    print('init criterion')
    criterion = nn.NLLLoss()
    # criterion = nn.MSELoss()
    
    print('starting iterations')
    
    for iter in tqdm(range(1, n_iters + 1)):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion, attn)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
training_pairs = [tensorsFromPair(random.choice(pairs))
                     for i in range(3)]

In [ ]:
par = training_pairs[1][1]


In [ ]:
tpair = training_pairs[0]
inp = tpair[0]
tar = tpair[1]
inp_size = inp.size(0)
tar_size = tar.size(0)
hidden_dim = 50
# print(tar_size)

# print(inp[0])

# print(vocab.n_words)

encoder = EncoderRNN(vocab.n_words,50)
encoder_hidden = encoder.initHidden()
encoder_outputs = torch.zeros(inp_size,hidden_size, device=device)


for i in range(inp_size):
   
    encoder_output, encoder_hidden = encoder(inp[i], encoder_hidden)
    encoder_outputs[i] = encoder_output[0, 0]
    

# print(encoder_outputs.shape)

decoder_input = torch.tensor([[SOS_token]], device=device)

#print(decoder_input)
decoder_hidden = encoder_hidden

# print(encoder_hidden[1].shape)

decoder = DecoderRNN(hidden_size,vocab.n_words)
loss  = 0
criterion = nn.NLLLoss()
# criterion = nn.MSELoss()
#criterion = nn.CrossEntropyLoss()

for i in range(tar_size):
    decoder_output, decoder_hidden= decoder(decoder_input, decoder_hidden)
    
    # print(decoder_output.view(-1).shape)
    # print(tar[i].shape)
    # print(weight_matrix[tar[i].item()].view(-1).shape)
    tar_tens = weight_matrix[tar[i].item()].view(-1).float()
    # print(tar_tens.shape)
    print(tar_tens)
    # print(tar_tens.type())
    # print(decoder_output.type())
    # print(tar_tens)
    print(decoder_output.shape)
    print(decoder_output)
    topv, topi = decoder_output.data.topk(1)
    print(vocab.index2word[topi.item()])
    print(topi.dtype)
    print(tar[i].dtype) 
    loss += criterion(decoder_output,tar[i])
    print(loss)
    
    decoder_input = tar[i]  # Teacher forcing


tensor([ 0.5139, -0.7162, -0.2702, -0.3961,  0.0777, -0.5102, -0.0960, -0.5193,
         1.1398, -0.4884,  0.5552, -0.0797,  0.6306, -0.1999, -0.7276, -1.1698,
        -0.7884, -0.1193, -0.0163,  0.5843, -0.6049,  0.3439, -0.1073,  0.3815,
         0.6044,  0.0985, -0.6297, -0.5533, -0.4024, -0.7582,  0.5817,  0.8636,
        -0.8008,  0.2618, -1.1834,  0.0997,  0.5694, -1.6251,  0.4834,  0.3614,
         0.7386, -0.6552, -0.3100, -0.6517,  0.5179,  0.3971,  0.2244,  0.1683,
         0.5761,  0.0467])
torch.Size([1, 402650])
tensor([[-12.8186, -12.7180, -12.8709,  ..., -12.9178, -12.8003, -12.8327]],
       grad_fn=<LogSoftmaxBackward0>)
actonel
torch.int64
torch.int64
tensor(12.8186, grad_fn=<AddBackward0>)
tensor([-0.3709,  0.4529, -0.0232,  0.4171, -0.4578, -0.4408, -0.1500,  0.2986,
        -0.3741, -0.3409,  0.3776,  0.0050,  0.3546,  0.8761, -0.1974,  0.0247,
        -0.3890,  0.9281,  0.0592,  0.8742,  0.5247,  0.2272,  0.2016, -0.3262,
         0.5885, -2.1577,  0.1074, -0.1838

In [ ]:
_, ind = decoder_output.data.topk(1)
lll = ind.item()
print(lll)
vocab.index2word[lll]

96408


'mousavian'

In [ ]:

word_indx = inp[1].item()
word = vocab.index2word[word_indx]
print(word)
print(vocab.glove[word])
print(weight_matrix[word_indx])


hidden = (torch.zeros(1, 1, 50),torch.zeros(1, 1, 50))
print(hidden[0].shape)
print(len(vocab.words))
embedded = nn.Embedding(len(vocab.words),50)
lstm = nn.LSTM(50,50)
embedded.from_pretrained(weight_matrix)
#lin = nn.Linear(hidden_size, tar_size)
softmax = nn.LogSoftmax(dim=1)

print(inp[1].item())
print(weight_matrix[inp[1].item()].view(-1))
print(vocab.index2word[inp[1].item()])
#print(vocab.glove['the'])
out = embedded(inp[1]).view(1,1,-1)
print(out)
print(out.shape)
out = F.relu(out)
output, hidden = lstm(out, hidden)
print(output.shape)
#output = softmax(output[0])
print(output.view(-1))



# out = embedded(inp[1]).view(1,1,-1)
# output, hidden = lstm(out, hidden)


#print(output.shape)

is
[ 6.1850e-01  6.4254e-01 -4.6552e-01  3.7570e-01  7.4838e-01  5.3739e-01
  2.2239e-03 -6.0577e-01  2.6408e-01  1.1703e-01  4.3722e-01  2.0092e-01
 -5.7859e-02 -3.4589e-01  2.1664e-01  5.8573e-01  5.3919e-01  6.9490e-01
 -1.5618e-01  5.5830e-02 -6.0515e-01 -2.8997e-01 -2.5594e-02  5.5593e-01
  2.5356e-01 -1.9612e+00 -5.1381e-01  6.9096e-01  6.6246e-02 -5.4224e-02
  3.7871e+00 -7.7403e-01 -1.2689e-01 -5.1465e-01  6.6705e-02 -3.2933e-01
  1.3483e-01  1.9049e-01  1.3812e-01 -2.1503e-01 -1.6573e-02  3.1200e-01
 -3.3189e-01 -2.6001e-02 -3.8203e-01  1.9403e-01 -1.2466e-01 -2.7557e-01
  3.0899e-01  4.8497e-01]
tensor([ 6.1850e-01,  6.4254e-01, -4.6552e-01,  3.7570e-01,  7.4838e-01,
         5.3739e-01,  2.2239e-03, -6.0577e-01,  2.6408e-01,  1.1703e-01,
         4.3722e-01,  2.0092e-01, -5.7859e-02, -3.4589e-01,  2.1664e-01,
         5.8573e-01,  5.3919e-01,  6.9490e-01, -1.5618e-01,  5.5830e-02,
        -6.0515e-01, -2.8997e-01, -2.5594e-02,  5.5593e-01,  2.5356e-01,
        -1.9612e+00, -

In [ ]:
def evaluate(encoder, decoder, sentence ,max_length=vocab.n_words, attn=False): # please check what should be max_length
    with torch.no_grad():
        input_tensor = tensorFromSentence(vocab, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(input_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            if attn:
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)
                decoder_attentions[di] = decoder_attention.data
            else:
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden) 
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('sos')
                break
            else:
                decoded_words.append(vocab.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

NameError: name 'vocab' is not defined

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        #print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0],attn=False)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 50
voc_size = len(vocab.words)
hidden_dim = 50
max_summary_size = 60
encoder1 = EncoderRNN(vocab.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, vocab.n_words).to(device)
#attn_decoder1 = AttnDecoderRNN(hidden_size, vocab.n_words, dropout_p=0.1).to(device)



In [ ]:
trainIters(encoder1, decoder1, 1500, print_every=100)

# trainIters(encoder1, attn_decoder1,1000,True, print_every=10)

1638142099.979296
encoder optimizer init
decoder optimizer init
choosing training pairs:1500
init criterion
starting iterations


  0%|          | 0/1500 [00:00<?, ?it/s]

2m 41s (- 37m 43s) (100 6%) 11.7590
5m 21s (- 34m 48s) (200 13%) 9.8065
7m 49s (- 31m 16s) (300 20%) 9.1782
10m 18s (- 28m 20s) (400 26%) 8.8418
12m 46s (- 25m 32s) (500 33%) 8.6845
15m 26s (- 23m 9s) (600 40%) 8.6397
18m 5s (- 20m 40s) (700 46%) 8.5531
20m 26s (- 17m 53s) (800 53%) 8.3687
22m 56s (- 15m 17s) (900 60%) 8.4153
25m 26s (- 12m 43s) (1000 66%) 8.1779
27m 56s (- 10m 9s) (1100 73%) 8.1868
30m 25s (- 7m 36s) (1200 80%) 7.9672
32m 48s (- 5m 2s) (1300 86%) 8.0229
35m 20s (- 2m 31s) (1400 93%) 8.0146
37m 46s (- 0m 0s) (1500 100%) 7.9211


In [142]:
attn = nn.Linear(hidden_dim * 2, max_summary_len)
# print(AttnDecoderRNN.embedded)
# print(AttnDecoderRNN.embedded[0])
print(AttnDecoderRNN.encoder_outputs.shape)
catt = torch.cat((AttnDecoderRNN.embedded[0], AttnDecoderRNN.hidden[0].view(1,-1)),1)
print(catt.shape)
attn_weights  = F.softmax(attn(catt), dim=1)
print(attn_weights.shape)
print(attn_weights.unsqueeze(0).shape)
print(encoder_outputs.shape)
print(encoder_outputs.unsqueeze(0).shape)
attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))

AttributeError: type object 'AttnDecoderRNN' has no attribute 'encoder_outputs'

In [143]:
import pickle

# save model and other necessary modules
all_info_want_to_save = {
    'encoder': encoder1,
    'decoder': decoder1,
    'vocab': vocab
}

with open("train_model.pkl","wb") as save_path:
    pickle.dump(all_info_want_to_save, save_path)




In [4]:
import pickle

# load the saved file
with open('train_model.pkl','rb') as ff:
    saved_info = pickle.load(ff)
    
# extract the information from the saved file
encoder1 = saved_info['encoder']
decoder1 = saved_info['decoder']

AttributeError: Can't get attribute 'EncoderRNN' on <module '__main__'>

In [6]:
evaluateRandomly(encoder1, decoder1)

NameError: name 'encoder1' is not defined